In [1]:
import pandas as pd

scores = pd.DataFrame({
    'name': ['Adam', 'Bob', 'Dave', 'Fred'],
    'age': [15, 16, 16, 15],
    'test1': [95, 81, 85, 90],
    'test2': [88, 70, 90, 90],
    'teacher': ['Ashby', 'Ashby', 'Jones', 'Jones']})
print(scores)

   name  age  test1  test2 teacher
0  Adam   15     95     88   Ashby
1   Bob   16     81     70   Ashby
2  Dave   16     85     90   Jones
3  Fred   15     90     90   Jones


1. Melt two numeric columns values into a single column. Add a new column to indicate what
the values mean.


In [2]:
melted = scores.melt(id_vars=['name', 'age'], value_vars=['test1', 'test2'])
melted = melted.assign(Mean=melted.value.mean())
print(melted)

   name  age variable  value    Mean
0  Adam   15    test1     95  86.125
1   Bob   16    test1     81  86.125
2  Dave   16    test1     85  86.125
3  Fred   15    test1     90  86.125
4  Adam   15    test2     88  86.125
5   Bob   16    test2     70  86.125
6  Dave   16    test2     90  86.125
7  Fred   15    test2     90  86.125


2. Un-melt the above.

In [3]:
melted.pivot_table(index=['name', 'age'], columns='variable', values='value').reset_index()

variable,name,age,test1,test2
0,Adam,15,95,88
1,Bob,16,81,70
2,Dave,16,85,90
3,Fred,15,90,90


3. Group by two columns, take the mean and unstack the result

In [13]:
import catboost as cb
import collections
import numpy as np

jb = pd.read_csv('datasets/2020-jetbrains-python-survey.csv', low_memory=False)

def get_unique_cols(jb):
    counter = collections.defaultdict(list)
    for col in sorted(jb.columns):
        period_count = col.count('.')
        if period_count >= 2:
            part_end = 2
        else:
            part_end = 1
        parts = col.split('.')[:part_end]
        counter['.'.join(parts)].append(col)
    unique_cols = []
    for cols in counter.values():
        if len(cols) == 1:
                unique_cols.extend(cols)
    return unique_cols

def prep_for_ml(df):
    # remove pandas types
    return (df
           .assign(**{col:df[col].astype(float)
                     for col in df.select_dtypes('number')},
                   **{col:df[col].astype('str').fillna('')
                     for col in df.select_dtypes(['object', 'category'])})
           )

def predict_col(df, col):
    df = prep_for_ml(df)
    missing = df.query(f'~{col}.isna()')
    cat_idx = []
    for i, typ in enumerate(df.drop(columns=[col]).dtypes):
        if str(typ) == 'object':
            cat_idx.append(i)
    X = missing.drop(columns=[col]).values
    y = missing[col]
    model = cb.CatBoostRegressor(iterations=20, cat_features=cat_idx)
    model.fit(X, y, cat_features=cat_idx)
    pred = model.predict(df.drop(columns=[col]))
    return df[col].where(~df[col].isna(), pred)

def tweak_jb(jb):
    unique_cols = get_unique_cols(jb)
    return (jb
            [unique_cols]
           .rename(columns=lambda c: c.replace('.', '_'))
           .assign(age=lambda df_: df_.age.str.slice(0, 2).astype(float)
                  .astype('Int64'),
                  are_you_datascientist=lambda df_: df_
                      .are_you_datascientist
                      .replace({'Yes': True, 'No': False, np.nan: False}),
                 # company_size=lambda df_: df_.company_size.replace({
                  #    'Just me': 1, 'Not sure': np.nan,  
                   #   'More than 5,000': 5000, '2-10': 2, '11-50': 11, ?!?!?!?!?!?!?!? what's wrong?
                    #  '51-500': 51, '501-1,000': 501,
                     # '1,001-5,001': 1001}).astype('Int64'),
                  country_live=lambda df_: df_.country_live
                      .astype('category'),
                  employment_status=lambda df_: df_.employment_status
                      .fillna('Other').astype('category'),
                  is_python_main=lambda df_: df_.is_python_main
                      .astype('category'),
                  team_size=lambda df_: df_.team_size
                      .str.split(r'-', n=1, expand=True)
                      .iloc[:, 0].replace('More than 40 people', 41)
                      .where(df_.company_size!=1, 1).astype(float),
                  years_of_coding=lambda df_: df_.years_of_coding
                      .replace('Less than 1 year', .5)
                      .str.extract(r'(\d+)').astype(float),
                  python_years=lambda df_: df_.python_years
                      .replace('Less than 1 year', .5)
                      .str.extract(r'(\d+)').astype(float),
                  python3_ver=lambda df_: df_.python3_version_most
                      .str.replace('_', '.').str.extract(r'(\d\.\d)')
                      .astype(float),
                  use_python_most=lambda df_: df_.use_python_most
                      .fillna('Unknown')
                  )
           .assign(team_size=lambda df_:predict_col(df_, 'team_size')
                  .astype(int))
           .drop(columns=['python2_version_most'])
           .dropna()
        )

jb2 = tweak_jb(jb)


Learning rate set to 0.5
0:	learn: 6.3373204	total: 162ms	remaining: 3.08s
1:	learn: 6.2416428	total: 187ms	remaining: 1.68s
2:	learn: 6.2042819	total: 214ms	remaining: 1.21s
3:	learn: 6.1713242	total: 240ms	remaining: 959ms
4:	learn: 6.1313697	total: 264ms	remaining: 793ms
5:	learn: 6.1163744	total: 288ms	remaining: 671ms
6:	learn: 6.1054688	total: 311ms	remaining: 577ms
7:	learn: 6.1040920	total: 331ms	remaining: 496ms
8:	learn: 6.0946976	total: 354ms	remaining: 433ms
9:	learn: 6.0906277	total: 372ms	remaining: 372ms
10:	learn: 6.0894286	total: 399ms	remaining: 327ms
11:	learn: 6.0757775	total: 430ms	remaining: 286ms
12:	learn: 6.0718265	total: 456ms	remaining: 246ms
13:	learn: 6.0696191	total: 480ms	remaining: 206ms
14:	learn: 6.0644258	total: 503ms	remaining: 168ms
15:	learn: 6.0638403	total: 516ms	remaining: 129ms
16:	learn: 6.0469462	total: 538ms	remaining: 95ms
17:	learn: 6.0442746	total: 562ms	remaining: 62.4ms
18:	learn: 6.0442742	total: 571ms	remaining: 30ms
19:	learn: 6.0384

In [19]:
jb2.groupby(['country_live', 'age']).mean(numeric_only=True).unstack()

nps_main_ide                                           \
age                     18        21        30         40        50   
country_live                                                          
Algeria          10.000000  7.500000  8.400000   8.000000       NaN   
Argentina        10.000000  8.743590  8.659091   8.857143  9.333333   
Armenia          10.000000  9.000000  9.666667        NaN       NaN   
Australia         8.500000  8.627119  8.846847   8.555556  8.600000   
Austria          10.000000  8.677419  8.843750   9.000000  8.750000   
...                    ...       ...       ...        ...       ...   
United States     9.113636  8.951592  8.911571   8.923711  8.656716   
Uruguay                NaN  9.500000  9.076923  10.000000       NaN   
Uzbekistan             NaN  9.750000       NaN        NaN       NaN   
Venezuela         7.500000  9.800000  7.800000   8.600000  7.500000   
Viet Nam         10.000000  9.000000  9.000000   9.000000       NaN   

                         python_years                                ...  \
age                   60           18        21        30        40  ...   
country_live                                                         ...   
Algeria         7.000000     1.000000  1.750000  2.800000  5.000000  ...   
Argentina      10.000000     3.000000  3.205128  4.795455  6.047619  ...   
Armenia              NaN     3.000000  2.769231  1.666667       NaN  ...   
Australia       9.400000     4.500000  4.966102  5.477477  6.809524  ...   
Austria              NaN     6.000000  2.838710  5.937500  7.272727  ...   
...                  ...          ...       ...       ...       ...  ...   
United States   9.231405     3.545455  3.970701  5.659454  6.373196  ...   
Uruguay              NaN          NaN  3.000000  5.076923  6.000000  ...   
Uzbekistan           NaN          NaN  1.500000       NaN       NaN  ...   
Venezuela            NaN     2.000000  2.700000  3.200000  3.800000  ...   
Viet Nam        8.000000     1.000000  2.178571  3.250000  3.000000  ...   

              years_of_coding                                  python3_ver  \
age                        30         40         50         60          18   
country_live                                                                 
Algeria              3.800000   6.666667        NaN   8.500000    3.650000   
Argentina            5.363636   8.904762  11.000000  11.000000    3.700000   
Armenia              5.000000        NaN        NaN        NaN    3.800000   
Australia            6.324324   8.920635  10.066667  10.500000    3.750000   
Austria              5.421875   7.772727  10.583333        NaN    3.800000   
...                       ...        ...        ...        ...         ...   
United States        6.083725   8.647423   9.921642  10.446281    3.777273   
Uruguay              6.307692   3.000000        NaN        NaN         NaN   
Uzbekistan                NaN        NaN        NaN        NaN         NaN   
Venezuela            6.200000  10.000000   8.500000        NaN    3.800000   
Viet Nam             3.500000   6.000000        NaN   1.000000    3.700000   

                                                                 
age                  21        30        40        50        60  
country_live                                                     
Algeria        3.750000  3.700000  3.766667       NaN  3.800000  
Argentina      3.758974  3.743182  3.733333  3.733333  3.800000  
Armenia        3.784615  3.766667       NaN       NaN       NaN  
Australia      3.740678  3.728829  3.720635  3.756667  3.780000  
Austria        3.729032  3.729687  3.745455  3.700000       NaN  
...                 ...       ...       ...       ...       ...  
United States  3.735924  3.733772  3.741856  3.741418  3.742975  
Uruguay        3.750000  3.746154  3.800000       NaN       NaN  
Uzbekistan     3.725000       NaN       NaN       NaN       NaN  
Venezuela      3.720000  3.680000  3.740000  3.800000       NaN  
Vie

4. Group by two columns, take the mean, and unstack the result, and flatten the columns.

In [22]:
def flatten_cols(df):
    cols = ['_'.join(map(str, vals))
        for vals in df.columns.to_flat_index ()]
    df.columns = cols
    return df

jb2.groupby(['country_live', 'age']).mean(numeric_only=True).unstack().pipe(flatten_cols)


,nps_main_ide_18,nps_main_ide_21,nps_main_ide_30,nps_main_ide_40,nps_main_ide_50,nps_main_ide_60,python_years_18,python_years_21,python_years_30,python_years_40,...,years_of_coding_30,years_of_coding_40,years_of_coding_50,years_of_coding_60,python3_ver_18,python3_ver_21,python3_ver_30,python3_ver_40,python3_ver_50,python3_ver_60
country_live,,,,,,,,,,,,,,,,,,,,,
Algeria,10.000000,7.500000,8.400000,8.000000,NaN,7.000000,1.000000,1.750000,2.800000,5.000000,...,3.800000,6.666667,NaN,8.500000,3.650000,3.750000,3.700000,3.766667,NaN,3.800000
Argentina,10.000000,8.743590,8.659091,8.857143,9.333333,10.000000,3.000000,3.205128,4.795455,6.047619,...,5.363636,8.904762,11.000000,11.000000,3.700000,3.758974,3.743182,3.733333,3.733333,3.800000
Armenia,10.000000,9.000000,9.666667,NaN,NaN,NaN,3.000000,2.769231,1.666667,NaN,...,5.000000,NaN,NaN,NaN,3.800000,3.784615,3.766667,NaN,NaN,NaN
Australia,8.500000,8.627119,8.846847,8.555556,8.600000,9.400000,4.500000,4.966102,5.477477,6.809524,...,6.324324,8.920635,10.066667,10.500000,3.750000,3.740678,3.728829,3.720635,3.756667,3.780000
Austria,10.000000,8.677419,8.843750,9.000000,8.750000,NaN,6.000000,2.838710,5.937500,7.272727,...,5.421875,7.772727,10.583333,NaN,3.800000,3.729032,3.729687,3.745455,3.700000,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
United States,9.113636,8.951592,8.911571,8.923711,8.656716,9.231405,3.545455,3.970701,5.659454,6.373196,...,6.083725,8.647423,9.921642,10.446281,3.777273,3.735924,3.733772,3.741856,3.741418,3.742975
Uruguay,NaN,9.500000,9.076923,10.000000,NaN,NaN,NaN,3.000000,5.076923,6.000000,...,6.307692,3.000000,NaN,NaN,NaN,3.750000,3.746154,3.800000,NaN,NaN
Uzbekistan,NaN,9.750000,NaN,NaN,NaN,NaN,NaN,1.500000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,3.725000,NaN,NaN,NaN,NaN
